<img src="https://files.cdn.thinkific.com/file_uploads/133292/images/d57/1b0/1a9/P03.jpg">

`MCSDSC02P03V3______`

`MCSDSC02P03V2______`

`MCSDSELIMC02P03V1130621`

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Проект принят! 

Если захочешь добавить его в портфолио, то наверное стоит почистить все общение и удалить лишний код. Для этого делай копию плз, потому что мы переизпользуем некоторые комментарии, чтобы не писать одинаковое)

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Есть что доработать. 

Как обычно, заголовки (комментариев) будут трех цветов. <font color="89DD00">Зеленый</font> — все правильно (и если комментария нет, то это также значит, что все правильно). <font color="FFB800">Желтый</font> — можно доработать. <font color="FF0F00">Красный</font> — нужно исправить.

## Постановка задачи

Заказчик исследования — страховая компания.  
Необходимо выяснить, как разные страховые случаи, возраст клиента и его доход влияют на статус выплаты страховой компенсации.

## План работ

1. Импортировать библиотеки, передать файл insurance.csv в датафрейм и проанализировать основную информацию о датасете.


2. Предобработать данные: 
    * Заполнить пропуски;
    * Заменить несоответствующие типы данных;
    * Заменить отрицательные значения в столбце с доходом;
    * Привести данные в столбцы с возрастной группой к единому регистру;
    * Удалить явные дубликаты.  


3. Выполнить стэмминг:
    * Посчитать уникальные значения в столбце с причинами страхового случая;
    * Выделить леммы и написать функцию, которая принимает основу слова, а возвращает все названия категорий с ним.


4. Произвести категоризацию данных:

    * Выделить словарь для возрастной группы.
    * Категоризировать причины выплат страховых платежей, статус выплаты, а также доход клиента.


5. Создать сводные таблицы с результатами, проанализировать их и ответить на вопросы, поставленные заказчиком:
    * есть ли зависимость между возрастной группой и статусом выплаты?
    * есть ли зависимость между причиной страхового случая и статусом выплаты?
    * есть ли зависимость между доходом клиента и статусом выплаты?

## Импорт данных и библиотек

Импортируем библиотеки, прочитаем файл `insurance.csv` и получим общую информацию о данных.

In [227]:
#! pip install nltk
#! pip install pymystem3

In [228]:
import pandas as pd
import numpy as np
import os
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

In [229]:
data_dir = '../../data'
#ins_df = pd.read_csv(os.path.join(data_dir, 'insurance.csv'))

ins_df = pd.read_csv('/Users/elimilo/Desktop/mcs_ds_02/data/insurance.csv')

In [230]:
ins_df.head(10)

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause
0,649,0.001,27384.0,4,Пожилой,NaN,0,0,0,повреждение автомобиля
1,81136,0.124,23735.0,4,пожилой,285140.0,0,0,0,упало дерево на автомобиль
2,70762,1.000,17170.0,3,Зрелый,-186030.0,0,0,0,сломала ногу в отпуске
3,53935,0.198,16068.0,3,Зрелый,-123540.0,0,0,0,хищение автомобиля (каско)
4,15476,0.041,10591.0,2,Молодой,-200020.0,1,0,0,несчастный случай в отпуске
5,64797,0.112,16065.0,3,Зрелый,NaN,0,0,0,врезались в автомобиль
6,67412,0.325,17167.0,3,ЗРЕЛЫЙ,388050.0,6,0,0,травма во время отдыха
7,44241,0.362,20085.0,4,ПОЖИЛОЙ,-99880.0,3,0,0,повреждение автомобиля
8,5069,0.115,23008.0,4,Пожилой,360040.0,0,0,0,сгорела квартира
9,16615,0.021,21906.0,4,Пожилой,510040.0,0,0,0,сгорел дом


In [231]:
ins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2301 non-null   int64  
 1   perc_of_compensation_paid       2301 non-null   float64
 2   age_in_days                     2301 non-null   float64
 3   age_id                          2301 non-null   int64  
 4   age_group                       2301 non-null   object 
 5   income                          2236 non-null   float64
 6   count_3-6_months_late           2301 non-null   int64  
 7   count_6-12_months_late          2301 non-null   int64  
 8   count_more_than_12_months_late  2301 non-null   int64  
 9   cause                           2301 non-null   object 
dtypes: float64(3), int64(5), object(2)
memory usage: 179.9+ KB


Столбцы содержат следующую информацию:

* id — ID держателя полиса;
* perc_of_compensation_paid — процент выплаченной компенсации;
* age_in_days — возраст держателя полиса в днях;
* age_id — ID возрастной группы;
* age_group — возрастная группа;
* income — доход держателя полиса;
* count_3-6_months_late — полис оформлен не позднее 3-6 месяцев;
* count_6-12_months_late — полис оформлен не позднее 6-12 месяцев;
* count_more_than_12_months_late — полис оформлен больше года назад;
* cause — причина страхового случая.

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Можно знать о множестве техник, но важно также уметь *выбирать* из них и понимать какая подойдет хорошо. Первые 10-15 строк + `info()` ответят на главные вопросы о структуре данных здесь.

`df` — это более абстрактное название для датафрейма, стоит называть предметно, чтобы было легче потом разобраться.

<h3><font color="#3c00ff">Update</font></h3>

удалил лишние команды, сменил название переменной на `ins_df`

### Выводы

* датасет содержит 2301 строк и 10 столбцов;
* столбцы id, age_id, count_... - целочисленные; perc_of_compensation_paid, age_in_days, income - дробные числа; age_group и cause - строковые;
* столбец id можно дропнуть, т.к. он содержит не существенную информацию;
* столбец income содержит nan и отрицательные числа => потребуется обработка;
* в столбце age_group есть записи в верхнем и нижнем регистре (при одинаковом ключе в age_id) => потребуется обработка.

In [232]:
ins_df = ins_df.drop(columns=['count_3-6_months_late', 'count_6-12_months_late',
                              'count_more_than_12_months_late'])

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

ID я бы удалять не стал, а вот срок годности полиса для анализа не нужен точно.

<h3><font color="#3c00ff">Update</font></h3>

удалил лишние столбцы

## Предобработка данных

**Обработка пропусков**

Проверим данные на наличие пропусков, просуммировав их.

In [233]:
ins_df.isna().sum()

id                            0
perc_of_compensation_paid     0
age_in_days                   0
age_id                        0
age_group                     0
income                       65
cause                         0
dtype: int64

Заполним пропуски в столбце `income`.

In [234]:
ins_df.loc[ins_df['age_id'] == 2,'income'] = ins_df.loc[
    ins_df['age_id'] == 2,'income'].fillna(ins_df[ins_df['age_id'] == 2]['income'].abs().mean())
ins_df.loc[ins_df['age_id'] == 3,'income'] = ins_df.loc[
    ins_df['age_id'] == 3,'income'].fillna(ins_df[ins_df['age_id'] == 3]['income'].abs().mean())
ins_df.loc[ins_df['age_id'] == 4,'income'] = ins_df.loc[
    ins_df['age_id'] == 4,'income'].fillna(ins_df[ins_df['age_id'] == 4]['income'].abs().mean())

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Круто, пропуски заполнены верно.

Если значения выборки равномерны и в ней нет выбросов, то среднее подойдет, но когда они есть — среднее значение может сильно сдвинуться вверх или вниз. Для такой выборки лучше использовать медиану. Это стоит учитывать.

Проверяем, что пропусков в `income` не осталось.

In [235]:
ins_df.isna().sum()

id                           0
perc_of_compensation_paid    0
age_in_days                  0
age_id                       0
age_group                    0
income                       0
cause                        0
dtype: int64

**Замена типа данных**

Проверяем, что в столбцах `age_in_days` и `income` вещественный тип данных.

In [236]:
ins_df[['age_in_days', 'income']].dtypes

age_in_days    float64
income         float64
dtype: object

Для этих столбцов нужен целочисленный тип, поэтому переведем их, а также избавимся от отрицательных значений в `income`.

In [237]:
ins_df[['age_in_days', 'income']] = ins_df[['age_in_days', 'income']].astype(int)
ins_df[['age_in_days', 'income']].dtypes

age_in_days    int64
income         int64
dtype: object

In [238]:
ins_df['income'] = ins_df['income'].abs()

**Поиск дубликатов с учетом регистра**

Проверим уникальные категории в столбце `age_group`.

In [239]:
ins_df['age_group'].unique()

array(['Пожилой', 'пожилой', 'Зрелый', 'Молодой', 'ЗРЕЛЫЙ', 'ПОЖИЛОЙ',
       'зрелый', 'молодой', 'МОЛОДОЙ'], dtype=object)

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Когда говорим о проверке уникальных — это больше про метод `unique()`. Не забудь тогда поправить описание шага.

<h3><font color="#3c00ff">Update</font></h3>

заменил на метод `unique()`

Приведем их к нижнему регистру.

In [240]:
ins_df['age_group'] = ins_df['age_group'].str.lower()
ins_df['age_group'].unique()

array(['пожилой', 'зрелый', 'молодой'], dtype=object)

**Обработка дубликатов**

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [241]:
ins_df.duplicated().sum()

7

In [242]:
ins_df = ins_df.drop_duplicates().reset_index(drop=True)

In [243]:
ins_df.duplicated().sum()

0

**Стэмминг**

Посчитаем уникальные значения в столбце `cause`.

In [244]:
russian_stemmer = SnowballStemmer('russian')

In [245]:
reasons_list = ins_df['cause'].unique().tolist()
print(reasons_list)
print()
print(len(reasons_list))

['повреждение автомобиля', 'упало дерево на автомобиль', 'сломала ногу в отпуске', 'хищение автомобиля (каско)', 'несчастный случай в отпуске', 'врезались в автомобиль', 'травма во время отдыха', 'сгорела квартира', 'сгорел дом', 'кража автомобиля', 'украли авто', 'потоп в квартире', 'украли багаж в отпуске', 'повреждение авто', 'проникли в жилье', 'кража авто', 'обокрали дачу', 'сотрясение во время отдыха', 'замкнуло электричество дома', 'несчастный случай на отдыхе', 'взломали квартиру', 'травма в отпуске', 'поломка авто', 'затопило дом', 'дтп', 'кража в квартире']

26


<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Можешь еще `value_counts()` тут использовать, в столбик поудобнее будет категории изучить. Или до этого так и было? В общем, посмотри что более информативно и удобно — "количество категорий" или "количество каждой категории".

<h3><font color="#3c00ff">Update</font></h3>

да, так и было))

In [246]:
russian_stemmer.stem('жилье')

'жил'

In [247]:
russian_stemmer.stem('авто')

'авт'

Выделим очевидные разные стеммы и виды страхования, которые из них вытекают: 
* авто - страхование автомобилей
* отдых  - страхование от несчастного случая
* отпуск  - страхование от несчастного случая
* квартира (-е) / (-у)  - страхование недвижимости
* дом  - страхование недвижимости
* дача (-у)  - страхование недвижимости
* жилье  - страхование недвижимости
* дтп  - страхование автомобилей

Напишем функцию, которая принимает основу слова и возвращает все названия категорий с ним. Сначала попробуем стэмминг и если не поймаем «чужих» названий — значит этот метод подойдет.

In [248]:
def search_keyword(categories, keyword):
    '''takes list, keyword and returns list of values, containing such keyword'''
    
    
    categories_list = []
    for category in categories:
        for word in category.split():
            stemmed_word = russian_stemmer.stem(word)
            if stemmed_word.startswith(keyword):
                categories_list.append(category)
    return categories_list

In [249]:
def search_keyword_var2(dataset, column, keyword):
    '''takes dataset, column, keyword and returns list of values, containing such keyword'''
    
    return dataset[dataset[column].str.contains(keyword)][column].unique().tolist()

In [250]:
search_keyword(reasons_list, 'авт')

['повреждение автомобиля',
 'упало дерево на автомобиль',
 'хищение автомобиля (каско)',
 'врезались в автомобиль',
 'кража автомобиля',
 'украли авто',
 'повреждение авто',
 'кража авто',
 'поломка авто']

In [251]:
search_keyword_var2(ins_df, 'cause', 'авто')

['повреждение автомобиля',
 'упало дерево на автомобиль',
 'хищение автомобиля (каско)',
 'врезались в автомобиль',
 'кража автомобиля',
 'украли авто',
 'повреждение авто',
 'кража авто',
 'поломка авто']

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Не забудь все-таки объяснить какие виды страхования из стемм вытекают.

По твоему вопросу — да, он отрезает окончание и оставляет только неизменяемую часть слова. В случае с 'авто', я думаю, букву 'о' он считает за соединительную гласную и тоже убирает ее.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

В Python для стемминг есть специальная библиотека NLTK

```
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')
```

которая содержит стеммеры (объекты, определяющие правила нахождения стемов).

В первую очередь тебе необходимо сформировать список текущих уникальных категорий с помощью методов `unique()` и `tolist()`.

Далее необходимо написать функцию, которая принимает основу слова, а возвращает все категории с ней. Ты сможешь добраться до нее с помощью двух циклов `for category in categories` и `for word in category` (когда будешь основу в категории, раздели с помощью метода `split()` все слова в ней).

Далее сохраняй stemmed_word с помощью русского стеммера и метода `stem()` (методу передавай итератор word) и проверяй с помощью условия проверяй равно ли stemmed_word той основе, которую ты передаешь функции.

Для нас важно, что ты понял как работает эта библиотека, поэтому задачу необходимо решить с помощью нее. 

Так же стоит подробнее объяснить выбор лемм. Дом, дача и жилье явно относятся к страхованию недвижимости, поэтому уже сейчас ты можешь обозначить, что при категоризации их можно объединить.

<h3><font color="#3c00ff">Update</font></h3>

вроде в целом работает, как надо. Единственное, стеммер превращает слово "авто" в "авт", а "жилье" в "жил". Соответственно, надо именно эти части слов передавать, чтобы правильно категории отобрать. Это так и должно работать? 

**Категоризация данных**

Cловарь — распространенный способ хранения информации, который помогает убрать текстовые параметры из основной таблицы и увеличить эффективность работы с данными. Выделим словарь для возрастных групп.

In [252]:
age_dict = ins_df[['age_id', 'age_group']].drop_duplicates().set_index('age_id').sort_index(ascending=True)
age_dict

,age_group
age_id,
2,молодой
3,зрелый
4,пожилой


<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Все так. Теперь текстовые значения age_group в основной таблице можно удалить и использовать только их индекс, что ускорит обработку данных и визуальную работу с таблицей.

Далее, опираясь на результат прошлого стэмминга, — категоризируем причины выплаты страховых платежей.

In [253]:
m = Mystem()

In [254]:
def lem(string):
    '''takes string and returns suitable category'''
    
    l = m.lemmatize(string)
    
    if ('автомобиль' in l) or ('дтп' in l):
        return 'автомобильное страхование'
    
    elif ('отпуск' in l) or ('отдых' in l):
        return 'страхование от несчастного случая'
    
    elif ('квартира' in l) or ('жилье' in l) or ('дача' in l) or ('дом' in l):
        return 'страхование недвижимости'
    
#    else:
#        return 'прочее'

In [ ]:
#ins_df['case_category'] = ins_df['cause'].apply(lem)
#ins_df['case_category'].unique()

In [ ]:
# проверка try-except
#ins_df.loc[2293, 'case_category'] = np.nan

In [258]:
# убрал 'авто'

try:
    ins_df['case_category'] = ins_df['cause'].apply(lem)
    ins_df['case_category'].sum()
except TypeError:
    print('Ошибка! В списке присутствует неизвестная категория!')
    ins_df['case_category'].fillna('неизвестная категория', inplace=True)
else:
    print('Все ок! Категории присвоены.')

Ошибка! В списке присутствует неизвестная категория!


In [259]:
ins_df['case_category'].value_counts()

страхование недвижимости             762
страхование от несчастного случая    656
автомобильное страхование            550
неизвестная категория                326
Name: case_category, dtype: int64

In [ ]:
#ins_df.loc[2293, 'case_category']

In [ ]:
# возврат значения
#ins_df.loc[2293, 'case_category'] = 'автомобильное страхование'

In [ ]:
#ins_df['case_category'].value_counts()

<h3><font color="#89DD00">Комментарий ассистента 3</font></h3>

Можешь по идее самой функцией try-except проверять и не заполнять отдельно nan. Применение категоризации `ins_df['case_category'] = ins_df['cause'].apply(lem)` тоже в сам блок try можно перенести.

<h3><font color="#FFB800">Комментарий ассистента 2</font></h3>

Решение с неизвестной категорией напоминает обработку ошибок конструкцией try-except. Имею в виду, что если в нее что-то попадает, то это уже как ошибка и результат неприемлемый: не может итоговая таблица быть с 'неизвестной категорией'.

Поэтому либо назвать ее 'остальные', либо придумать как try-except здесь использовать. На счет последнего — какую операцию мы не можем проделать со столбцом, если в нем есть NaN? Вот ее и можно попытаться проделать после заполнения 'case_category'. Если возникает ошибка, то пускай NaN заполняются как 'неизвестная категория'.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Для решения этот задачи есть библиотека pymystem3

```
from pymystem3 import Mystem
m = Mystem()
```

которая в отличие от NLTK выдает *список* лемматизированных слов (сведенных к словарной форме), а не строку. 

Для этого необходимо написать функцию лемматизации

```
def lem(row):
    l = m.lemmatize(row)
    if 'word_1' in l or 'word_2' in l:
        return 'new_category'
    ...
```

и применить ее через метод `apply()` к столбцу с причинами

```
df['column_1'] = df['column_1'].apply(lem)
```

Так же обрати внимание, что некоторые твои категории, например, "дтп" гипотетически могли встречаться всего 2 или 3 раза. Этих данных недостаточно, чтобы строить какие-то выводы, именно поэтому мы проводим категоризацию. У нас получилось выделить 3 вида страхования из тех формулировок, которые представлены в таблице, поэтому ты тоже подумай еще.

<h3><font color="#3c00ff">Update</font></h3>

переделал

Категоризируем статус выплаты компенсации — 0 выплат, выплачена часть, полностью выплачена.

In [ ]:
def comp_status(x):
    if x == 1:
        return 'полностью выплачена'
    elif x == 0:
        return 'нет выплат'
    elif 0 < x < 1:
        return 'выплачена часть'

In [ ]:
ins_df['compensation_status'] = ins_df['perc_of_compensation_paid'].apply(comp_status)
ins_df['compensation_status'].unique()

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Все так.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Здесь мы ожидаем увидеть применение функции категоризации выплаты через метод `apply()`. Прочитай, пожалуйста, про него в конспекте по категоризации и переделай. Внутри функции у тебя должно быть простое условие: если выплачено 100%, то возращай 'полностью выплачена'. 

Задавай вопросы в ветку категоризации, если будет что-то непонятно.

<h3><font color="#3c00ff">Update</font></h3>

переделал

In [ ]:
ins_df['compensation_status'].value_counts()

Категоризируем `income`, разбив данные по квантилям.

In [ ]:
ins_df.income.quantile(([0.25, 0.50, 0.75]))

In [ ]:
ins_df['income_level'] = pd.qcut(x=ins_df['income'], q=4, labels=['низкий доход', 'доход ниже среднего', 'доход выше среднего', 'высокий доход'])

In [ ]:
ins_df = ins_df.drop(columns=['perc_of_compensation_paid', 'age_group', 'age_in_days', 'income', 'cause'])
ins_df

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Ок.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Ты обозначаешь границы, а есть еще метод qcut. Он принимает столбец, по которому категоризируем, количество корзин и лэйблы. С ним у тебя каждая группа должна получится одинаковой размерности. Он все упростит, переделай с ним.

<h3><font color="#3c00ff">Update</font></h3>

переделал

## Результаты

Подготовим 3 таблицы и изучим результат.

In [ ]:
table_1 = ins_df.pivot_table(
    values='id',
    columns='age_id',
    index='compensation_status',
    aggfunc='count'
    ).set_axis(age_dict['age_group'], axis=1).reindex(
    index=['нет выплат', 'выплачена часть', 'полностью выплачена'])

table_1

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Да, отличная вещь с описанием методов. Добавили в задачи

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Круто, ты самостоятельно разобрался с аргументами `pivot_table()`. У нас результаты незначительно отличаются, но это из-за разной категоризации. 

И можешь еще наш конспект по изменению типов данных посмотреть, там тоже есть кратко об аргументах `pivot_table()`.

<h3><font color="#3c00ff">Update</font></h3>

переделал со словариком)  
конспекты почитал, спасибо)
кстати, возможно это ни для кого больше не секрет, но я прямо открыл для себя очень удобные hotkeys - shift и shift+tab для вызова контекстного списка с методами и справки по методу) предлагаю добавить в контекст по jupyter

In [ ]:
#table_1.plot(kind='line', grid=True, subplots=False, figsize=(15,5), logy=True, xlabel='статус выплаты',
             #title='Логарифмическое распределение статуса выплат в зависимости от возраста клиента')

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Круто, что внедряешь здесь элементы исследовательского анализа данных. Этому будет посвящен новый Курс 03 и его проект. Если что, на данном этапе нам будет достаточно сводной таблицы выше

*ответьте на вопрос — есть ли зависимость между возрастной группой и статусом выплаты?*

### вывод ###
* вместе с возрастом клиентов растет и общее количество страховых случаев - это можно объяснить тем, что в зрелом и пожилом возрасте люди больше задумаются о страховании своих рисков. Кроме того, можно предположить, что в зрелом возрасте люди больше берут существенные суммы кредитов(авто, ипотека), при этом банки требует стрхование объекта кредита, а также жизни и здоровья заемщиков;
* клиентам в зрелом возрасте в большей степени достаются частичные выплаты, чем пожилым клиентам;
* пожилым клиентам в большей степени достаются полные выплаты, чем клиентам в зрелом возрасте.

In [ ]:
table_2 = ins_df.pivot_table(
    values='id',
    index='case_category',
    columns='compensation_status',
    aggfunc='count').reindex(columns=['нет выплат', 'выплачена часть', 'полностью выплачена'])

table_2

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Все так.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Как и говорил выше, из причин обращения нужно категоризировать 3 вида страхования.

<h3><font color="#3c00ff">Update</font></h3>

переделал категории, обновил выводы

In [ ]:
#table_2.plot(kind='line', grid=True, subplots=False, figsize=(15,5), logy=True, xlabel='статус выплаты',
             #title='Логарифмическое распределение статуса выплат в зависимости от категории случая')

*есть ли зависимость между причиной страхового случая и статусом выплаты?*

## вывод ##
* заметно, что в автостраховании чаще делают полные и частичные выплаты, чем в других категориях
* в страховании от НС меньше отказов в выплатах, чем в других категориях

In [ ]:
table_3 = ins_df.pivot_table(
    values='id',
    index='income_level',
    columns='compensation_status',
    aggfunc='count').reindex(columns=['нет выплат', 'выплачена часть', 'полностью выплачена'])

table_3

In [ ]:
#table_3.plot(kind='line', grid=True, subplots=False, figsize=(15,5), logy=True, xlabel='уровень дохода',
             #title='Логарифмическое распределение статуса выплат в зависимости от дохода клиентов')

*есть ли зависимость между доходом клиента и статусом выплаты?*

## вывод ##
* людям с низким доходом в этой выборке полностью выплачивают компенсации чаще остальных
* при этом им также чаще отказывают в выплатах
* возможно, это погрешности выборки, требуется доп. исследование

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Норм.

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Из последнего столбца можно заметить, что людям с низким доходом в этой выборке полностью выплачивают компенсации чаще остальных. Можно отметить это

<h3><font color="#3c00ff">Update</font></h3>

обновил выводы